In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%%capture
!pip install bertopic
!pip install -U kaleido

In [3]:
import re
import pandas as pd
from datetime import datetime



In [4]:
# Load data
ny_post = pd.read_csv('/content/gdrive/MyDrive/Cendie/ny_post(1).csv')


In [5]:
ny_post.Fecha

0      2020-03-01 00:00:00
1      2020-03-01 00:00:00
2      2020-03-01 00:00:00
3      2020-03-01 00:00:00
4      2020-03-01 00:00:00
              ...         
646    2020-10-25 00:00:00
647    2020-10-25 00:00:00
648    2020-10-25 00:00:00
649    2020-10-25 00:00:00
650    2020-10-25 00:00:00
Name: Fecha, Length: 651, dtype: object

In [6]:
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
ny_post.Texto = ny_post.apply(lambda row: re.sub(r"http\S+", "", row.Texto).lower(), 1)
ny_post.Texto= ny_post.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.Texto.split())), 1)
ny_post.Texto = ny_post.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.Texto).split()), 1)
ny_post_texto = ny_post.Texto.to_list()

In [8]:
nypost_texto= []
for i in range(len(ny_post_texto)):
  a = ny_post_texto[i].replace("sign up for our special edition newsletter to get a daily update on the coronavirus pandemic ", "")
  pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
  text = pattern.sub('', a)
  #nytimes_texto.append(text) 
  nypost_texto.append(text)

In [9]:
nypost_texto

['new york city officials urging folks panic first confirmed case coronavirus state sunday shortly big apple case announced mayor bill de blasio released statement saying authorities fully prepared respond development beginning said matter would positive case coronavirus new york mayor said health authorities state high alert weeks fully prepared respond statement said continue ensure new yorkers facts resources need protect new york city council speaker corey johnson tweeted cause panic council hold hearing thursday find big apple prepared outbreak johnson said governor andrew cuomo sunday evening announced first case deadly virus state woman late recently traveled iran isolated monitored home manhattan patient sick enough hospitalized according nyc department health mental hygiene city health officials identified close contacts woman may exposed take appropriate measures prevent spread covid health commissioner dr oxiris barbot said statement health department say many people patient

In [10]:
nypost_text = pd.Series(nypost_texto)
 


In [11]:
nypost_text

0      new york city officials urging folks panic fir...
1      even big apple beloved chinese restaurants cri...
2      coronavirus claimed second life us man died se...
3      first case coronavirus new york city confirmed...
4      syracuse university student eli schwemler ente...
                             ...                        
646    white house chief staff mark meadows sunday de...
647    retired usually take seasonal retail work extr...
648    white house chief staff mark meadows said sund...
649    nfl finally punished tennessee titans team cov...
650    florida nurse claimed doctor intentionally inf...
Length: 651, dtype: object

In [17]:
from bertopic import BERTopic
topic_model = BERTopic(language="english", calculate_probabilities = True)#, min_topic_size=10, nr_topics=14 )#, verbose=True)
topics, probs = topic_model.fit_transform(nypost_text)

In [15]:
freq = topic_model.get_topic_info(); freq.head(30)

,Topic,Count,Name
0,-1,201,-1_coronavirus_city_cases_virus
1,0,105,0_trump_biden_coronavirus_americans
2,1,83,1_players_game_league_games
3,2,66,2_restaurant_restaurants_coronavirus_bar
4,3,49,3_hospital_patients_covid_virus
5,4,25,4_percent_york_cuomo_deaths
6,5,22,5_students_school_campus_schools
7,6,17,6_china_lab_chinese_zoo
8,7,17,7_city_homeless_blasio_mta
9,8,16,8_italy_milan_italian_coronavirus


In [18]:
for i in range(0, 14):    
    topic_nr = freq.iloc[i]["Topic"]
    print(f'Tópico {i-1}:')
    print(topic_model.get_topic(topic_nr))    # You can select a topic number as shown above

Tópico -1:
[('city', 0.017524176856767125), ('coronavirus', 0.016653104951492446), ('york', 0.013067382664196412), ('covid', 0.012876155780737182), ('virus', 0.01247551336058308), ('cuomo', 0.011138397433763396), ('pandemic', 0.009199947473879835), ('patients', 0.0087084799882163), ('public', 0.007931075826166116), ('us', 0.007832238565659363)]
Tópico 0:
[('biden', 0.025100284486952645), ('coronavirus', 0.01860281055997431), ('americans', 0.013169196947914035), ('pandemic', 0.013124077339167968), ('vaccine', 0.012394765524783503), ('campaign', 0.011536592230158385), ('democrats', 0.010883339080291324), ('virus', 0.010853652286123115), ('tested', 0.010206160342273568), ('testing', 0.009446911649425562)]
Tópico 1:
[('players', 0.042437526311159705), ('season', 0.030441807434164046), ('game', 0.03031356269002323), ('league', 0.02345430054376299), ('player', 0.01720384520015784), ('yankees', 0.016677865409961502), ('mets', 0.016619571434146233), ('baseball', 0.01476806540078827), ('mlb', 0

In [19]:
fig = topic_model.visualize_hierarchy()
fig.update_layout(title_text = 'Dendograma - Ny post')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_post/Dendograma.jpg")
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_post/Dendograma.html")

In [20]:
fig = topic_model.visualize_barchart([0,1,2,3,4,5], n_words = 10)
fig.update_layout(title_text = 'Tópicos - Ny Post')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_post/score-topics(1).jpg")
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_post/score-topics(1).html")
fig

In [21]:
fig = topic_model.visualize_barchart([6,7,8,9,10,11], n_words = 10)
fig.update_layout(title_text = '')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_times/score-topics(2).jpg")
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_post/score-topics(2).html")
fig

In [22]:
fig = topic_model.visualize_barchart([12], n_words = 10)
fig.update_layout(title_text = '')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_post/score-topics(3).jpg")
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_post/score-topics(3).html")
fig

In [23]:
fig = topic_model.visualize_heatmap()
fig.update_layout(title_text = 'Matriz de Similaridad - Ny post')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_post/similaridad.jpg")
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_post/similaridad.html")
fig

In [24]:
topic_model.visualize_term_rank()

In [27]:
fig = topic_model.visualize_topics(); fig

In [26]:
topics_over_time = topic_model.topics_over_time(docs=nypost_text, 
                                                topics=topics, 
                                                timestamps=ny_post.Fecha, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

In [28]:
fig = topic_model.visualize_topics_over_time(topics_over_time)
fig.update_layout(title_text = 'Tópicos a través del tiempo - Ny Post')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_post/overtime.jpg")
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_post/overtime.html")
fig

In [ ]:
probabilidades

0    1
dtype: int64

In [ ]:
probabilidades = []
for i in range(0, 651):
  proba = pd.Series(probs[i].argmax())
  proba = proba[0]
  probabilidades.append(proba)

print(probabilidades)
  



[1, 4, 5, 2, 7, 11, 5, 4, 5, 0, 10, 2, 0, 10, 5, 14, 5, 14, 4, 5, 5, 13, 2, 5, 2, 10, 14, 7, 1, 8, 7, 8, 0, 5, 10, 2, 10, 8, 4, 4, 1, 2, 1, 2, 2, 2, 0, 8, 7, 0, 12, 1, 5, 0, 2, 13, 13, 5, 10, 4, 2, 1, 8, 3, 0, 0, 4, 3, 1, 2, 13, 7, 3, 4, 8, 0, 1, 0, 1, 7, 7, 1, 13, 1, 13, 1, 0, 12, 2, 1, 8, 8, 0, 3, 1, 1, 1, 2, 2, 2, 1, 2, 1, 3, 2, 2, 2, 1, 14, 0, 10, 3, 2, 12, 8, 1, 2, 14, 13, 1, 1, 7, 3, 2, 0, 0, 7, 0, 1, 3, 9, 2, 13, 2, 3, 2, 10, 0, 4, 5, 3, 6, 5, 10, 2, 1, 2, 1, 11, 2, 7, 5, 4, 2, 2, 1, 1, 2, 1, 1, 0, 2, 0, 9, 2, 8, 1, 1, 4, 4, 12, 11, 0, 14, 8, 11, 2, 13, 2, 1, 11, 6, 10, 2, 2, 4, 1, 3, 8, 2, 4, 2, 1, 0, 13, 2, 2, 13, 12, 1, 6, 2, 13, 2, 1, 1, 1, 8, 14, 3, 8, 4, 0, 11, 3, 1, 1, 7, 1, 3, 0, 14, 3, 10, 6, 7, 1, 4, 1, 0, 11, 12, 8, 2, 10, 0, 1, 1, 10, 1, 0, 1, 7, 5, 6, 8, 1, 0, 4, 1, 2, 1, 2, 1, 0, 1, 2, 0, 14, 5, 0, 11, 0, 0, 0, 6, 1, 2, 13, 1, 6, 3, 13, 1, 1, 1, 2, 1, 0, 3, 11, 8, 12, 4, 0, 10, 0, 2, 1, 12, 4, 13, 2, 4, 11, 8, 4, 0, 8, 2, 0, 8, 2, 6, 0, 14, 14, 1, 0, 0, 2, 4, 13, 2

In [ ]:
probabilidades = pd.Series(probabilidades, name = 'Topico más probable')


In [ ]:
probabilidades

0       1
1       4
2       5
3       2
4       7
       ..
646     0
647    12
648     0
649     9
650     2
Name: Topico más probable, Length: 651, dtype: int64

In [ ]:
ny_post_contopicos = pd.concat([ny_post, probabilidades], axis = 1)
ny_post_contopicos =  ny_post_contopicos[[ 'Titulo', 'Url', 'Fecha' ,'Texto' ,'Topico más probable']]
ny_post_contopicos

,Unnamed: 0,Titulo,Url,Fecha,Texto,Topico más probable
0,0,NYC officials: Don’t panic after first confirm...,https://nypost.com/2020/03/01/nyc-officials-do...,2020-03-01 00:00:00,new york city officials are urging folks not t...,1
1,1,Coronavirus taking a toll on Chinese restauran...,https://nypost.com/2020/03/01/coronavirus-taki...,2020-03-01 00:00:00,even the big apple s most beloved chinese rest...,4
2,2,Second US coronavirus death reported in Washin...,https://nypost.com/2020/03/01/second-us-corona...,2020-03-01 00:00:00,the coronavirus has claimed a second life in t...,5
3,3,First case of coronavirus confirmed in New Yor...,https://nypost.com/2020/03/01/first-case-of-co...,2020-03-01 00:00:00,the first case of coronavirus in new york city...,2
4,4,NY study-abroad students in Italy being sent h...,https://nypost.com/2020/03/01/ny-study-abroad-...,2020-03-01 00:00:00,syracuse university student eli schwemler ente...,7
...,...,...,...,...,...,...
646,646,Mark Meadows defends Pence campaigning after s...,https://nypost.com/2020/10/25/meadows-defends-...,2020-10-25 00:00:00,white house chief of staff mark meadows on sun...,0
647,647,Will I be passed over for work since I’m 65 an...,https://nypost.com/2020/10/25/will-i-be-passed...,2020-10-25 00:00:00,i m retired and and usually take on seasonal r...,12
648,648,Mark Meadows says ‘we’re not going to control’...,https://nypost.com/2020/10/25/wh-chief-of-staf...,2020-10-25 00:00:00,sign up for our special edition newsletter to ...,0
649,649,NFL mulling Raiders punishment after Titans ge...,https://nypost.com/2020/10/25/nfl-mulls-raider...,2020-10-25 00:00:00,sign up for our special edition newsletter to ...,9
